<a href="https://colab.research.google.com/github/Afififaiz/Day2-AiNusantara/blob/main/Day2_AiNusantara_Mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Import Necessary Libraries
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np